In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedLineDocument
from nltk.tokenize import word_tokenize
import pandas as pd
import spacy 
nlp = spacy.load("de_core_news_sm")
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import nltk
from nltk.corpus import wordnet as wn
from langdetect import detect
from sklearn.manifold import TSNE

In [2]:
def generate_random_vector(dimensions,seed):
    arr = np.array([0] * (dimensions-seed) + [1] * int(seed/2) + [-1] * int(seed/2) )
    np.random.shuffle(arr)
    return(arr)

In [17]:
import csv
import re
textz=[]
word_vec=dict()
word_docz=dict()
dimensions=200
seed=4

with open('../data/after_nltk/df_content.csv', newline='', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile) 
    for row in reader:
        textz.append(row['content_nltk'].lower())
        #if int(row['id'])>5000:
        #    break
        if int(row['id'])%1000==0:
            print(row['id'])
        current_doc_wordz=dict()
        content=row['content_nltk'].lower()
        for word in re.split('[\W]', content):
            if word not in current_doc_wordz:
                current_doc_wordz[word]=1
                try:
                    word_docz[word]=word_docz[word]+1
                except:
                    word_docz[word]=1
                    word_vec[word]=generate_random_vector(dimensions,seed)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [18]:
nonsingular_wordz = dict((k, v) for k, v in word_docz.items() if v > 1)

In [19]:
normalize_factor=1/sum(nonsingular_wordz.values())
for k in nonsingular_wordz:
    nonsingular_wordz[k]*=normalize_factor

In [20]:
sum(nonsingular_wordz.values())

0.9999999999994301

In [21]:
len(textz)

18143

In [22]:
doc_vec=np.ones((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            doc_vec[line_id] = ( word_vec[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
    line_id=line_id+1

<i>Instead of doc_vec[text_id] = doc_vec[text_id] + word_vec[word] You will simply have a SECOND PASS loop in which You'll do something like word_vec[word] = word_vec[word] + doc_vec[doc_id]</i>

In [ ]:
# returning the document vectors back into wordvectors
word_vec_loop =()
for word in word_vec:
    word_vec_loop[word] = word_vec[word] + doc_vec[line_id]

In [ ]:
# rerunning the loop with new word_vectors
doc_vec=np.ones((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            doc_vec_loop[line_id] = ( word_vec_loop[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
    line_id=line_id+1

In [10]:
from scipy.spatial import distance

def nearest_neighbor(query_id,database):
    query=database[query_id]
    closest = distance.cdist([query], database, 'cosine')
    return closest

In [23]:
import numpy.ma as ma

query_id=420
nns=nearest_neighbor(query_id,doc_vec)
nn_id=np.argmin(ma.masked_where(nns<0.0000000001, nns))
print(query_id,textz[query_id])
print(nn_id,nns[0][nn_id],textz[nn_id])

420 welt lernen nie vergessen schopenhauer schopenhauers werk gehoert klassisch repertoire philosophie revolutionieren traditionell menschenbild abendlandes eroeffnete neue horizonte fragens forschens spierlings schopenhauerlexikon leben werk kolossal weltverneiners amuesant lehrreich einzeln stichwoertern aufbereiten afterweisheit ueber stachelschweine tod teufel fundgrube sowohl fuer einsteiger fuer kenner
3830 0.5571404629775837 handbuch geben berblick ueber schopenhauers leben praesentiert wichtig werke werkgruppen legen philosophisch kontext dar entstehen ausfuehrlich wirkung philosophie literatur musik bildend kunst darstellen besonder hilfsmittel forschung bieten handbuch zudem seitenkonkordanz gaengigen werkausgaben schopenhauers beruecksichtigtdie auflage werden aktualisieren artikel anthropologie evolutionstheorie geometrie tierethik ergaenzt


<h1>Compute Cosine Similarity through dot product</h1>

In [12]:
from sklearn.preprocessing import Normalizer
nrm = Normalizer('l2')

doc_vec_normed = nrm.fit_transform(doc_vec)

In [13]:
df = pd.read_pickle('../data/after_nltk/data_after_nltk.pkl')

In [24]:
# choose example 
example = query_id
title_1 = df['title'].iloc[example]
abstract_1 = textz[example]
# get example vector
vec_1 = doc_vec_normed[example,:]

# compute similarity
sim = np.dot(doc_vec_normed[example+1:,:],vec_1)

# find most similar document, see paper
title_2 = df['title'].iloc[np.argmax(sim)+example+1]
abstract_2 = textz[np.argmax(sim)+example+1]

print('\nSimilarity:')
print(np.max(sim))
print('\nTitle 1:')
print(title_1)
print('\nAbstract 1:')
print(abstract_1)
print('\nTitle 2:')
print(title_2)
print('\nAbstract 2:')
print(abstract_2)


Similarity:
0.4388970199029248

Title 1:
 Kleines Schopenhauer-Lexikon  / von Volker Spierling

Abstract 1:
welt lernen nie vergessen schopenhauer schopenhauers werk gehoert klassisch repertoire philosophie revolutionieren traditionell menschenbild abendlandes eroeffnete neue horizonte fragens forschens spierlings schopenhauerlexikon leben werk kolossal weltverneiners amuesant lehrreich einzeln stichwoertern aufbereiten afterweisheit ueber stachelschweine tod teufel fundgrube sowohl fuer einsteiger fuer kenner

Title 2:
 Sinn  / von Gregor Bongaerts

Abstract 2:
studierend soziologie leserinnen allgemein geistes sozialwissenschaftlicher theoriebildung interessieren finden buch orientierung nahezu unuebersichtlichen gelaende soziologisch theorien sinnbegriffe vergleichend bersicht stellen einfuehrung zentrale sinnbegriffe basiskategorien verschieden soziologisch theorien verwenden abhaengigkeit konstruktion soziologisch gegenstandsbereichs theoretisch grundannahmen weisen beobachtbar z

<h1>K nearest Neighbors</h1>

In [25]:
doc_vec_normed.shape

(18143, 200)

In [26]:
# compares similarity of two abstracts through dot product

# validation_vector1 
validation_vector1 = query_id
val_abstract_1 = textz[validation_vector1]
# get validation_vector1
val_vec_1 = doc_vec_normed[validation_vector1,:]

# compute similarity of all entities
vec_temp = []
for vec in doc_vec_normed:
    i = np.dot(val_vec_1,vec)
    vec_temp.append(i)

# turn list into DataFrame
df_vec_temp = pd.DataFrame(vec_temp)

# Create new DataFrame for sorted sim-values
df_vec_temp_sorted = df_vec_temp.copy()
df_vec_temp_sorted.sort_values(by=[0],inplace = True, ascending=False)

# Get list of indices
lst_vec_temp_sorted = df_vec_temp_sorted.iloc[1:11,:].index.values.tolist()
lst_vec_temp_sorted

print(df['title'].iloc[validation_vector1])
print('\n')

for i in lst_vec_temp_sorted:
#    print('\n')
    print(df_vec_temp.iloc[i].values)
    print(df['title'].iloc[i])
#     print(textz[i])
#     print('\n')

 Kleines Schopenhauer-Lexikon  / von Volker Spierling


[0.43889702]
 Sinn  / von Gregor Bongaerts
[0.38823975]
 Schopenhauer-Handbuch  : Leben – Werk – Wirkung 
[0.36922715]
 Kunst, Raum, Autorschaft  : Der Nachlass des US-amerikanischen Malers C.H. Phillips (1889-1975) aus autorgeografischer Perspektive  / von Alexia Pooth
[0.34487528]
 Angewandte Funktionalanalysis  : Funktionalanalysis, Sobolev-Räume und elliptische Differentialgleichungen  / von Manfred Dobrowolski
[0.3145693]
 Einführung in die Geschichte der Soziologie  / von Hermann Korte
[0.28757217]
 Was ist Literatur?  : Basistexte zur Literaturtheorie 
[0.27820936]
 Bildbegriff und Kunstverständnis im kunstpädagogischen Kontext  / von Kunibert Bering, Julia Gerber, Nadja Nafe, Rolf Niehoff, Karina Pauls
[0.27285277]
 Kompendium Adoleszenzpsychiatrie  : Krankheitsbilder mit CME-Fragen 
[0.27210431]
 Der Einfluss der Erstsprache auf den Erwerb der Zweitsprache  : Eine empirische Untersuchung zum Einfluss erstsprachlicher Stru

In [ ]:
#np.save('../data/after_nltk/doc_vec-200.npy', doc_vec)

<h1>Step 2: Generate our 2D embedding</h1>
https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne

In [ ]:
doc_vec = np.load('../data/after_nltk/doc_vec-200.npy')

In [ ]:
# Creates TSNE model and plots it
x = []
y = []


def tsne_plot(message_embeddings):
    labels = []
    tokens = []  
    
    tsne_model = TSNE(perplexity=50, n_components=2, init='pca',
                      n_iter=5000, 
                      random_state=23)

    new_values = tsne_model.fit_transform(message_embeddings)

    global x,y
    
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i],s=0.2)
    plt.show()


In [ ]:
tsne_plot(doc_vec)

In [ ]:
df_values = pd.DataFrame.from_dict({'x':x,'y':y})

In [ ]:
print(df_values)

In [ ]:
df = pd.read_pickle('../data/after_nltk/data_after_nltk.pkl')
df_all = pd.concat([df, df_values], axis=1, sort=False)
df_all = df_all.drop(['Languagereview', 'content_nltk'], axis=1)
df_all.to_excel('../data/after_nltk/data_for_tableau_testing_p40.xlsx', index = False)